In [186]:
import re
from pathlib import Path

import numpy as np

In [177]:
input_data = Path("example.txt").read_text().strip()
print(input_data)

0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2


In [178]:
shape_pattern = re.compile(r"(\d+:\n(?:[#.]+\n)+)", re.MULTILINE)
shape_strings = shape_pattern.findall(input_data)
shape_strings

['0:\n###\n##.\n##.\n',
 '1:\n###\n##.\n.##\n',
 '2:\n.##\n###\n##.\n',
 '3:\n##.\n###\n##.\n',
 '4:\n###\n#..\n###\n',
 '5:\n###\n.#.\n###\n']

In [179]:
shape_dict = {}
for shape_string in shape_strings:
    index, shape_array_string = shape_string.split(":")
    index = int(index)
    shape_lines = shape_array_string.strip().splitlines()
    shape_array = np.array(
        [[char == "#" for char in line] for line in shape_lines],
    )
    shape_dict[index] = shape_array

shape = np.array(list(shape_dict.values()), dtype=bool)
print(shape.astype(int))

[[[1 1 1]
  [1 1 0]
  [1 1 0]]

 [[1 1 1]
  [1 1 0]
  [0 1 1]]

 [[0 1 1]
  [1 1 1]
  [1 1 0]]

 [[1 1 0]
  [1 1 1]
  [1 1 0]]

 [[1 1 1]
  [1 0 0]
  [1 1 1]]

 [[1 1 1]
  [0 1 0]
  [1 1 1]]]


In [185]:
requirement_pattern = re.compile(r"(\d+x\d+): ([\d ]+)")
region_requirements = []
shape_quantity_requirements = []
requirement_pattern.findall(input_data)
for match in requirement_pattern.findall(input_data):
    size_str, counts_str = match
    width, height = map(int, size_str.split("x"))
    counts = np.array(counts_str.split(), dtype=int)
    region_requirements.append(np.zeros((height, width), dtype=int))
    shape_quantity_requirements.append(counts)

shape_quantity_requirements = np.array(shape_quantity_requirements, dtype=int)

for i, (region, shape_quantity) in enumerate(
    zip(region_requirements, shape_quantity_requirements, strict=True),
):
    print(
        f"Region {i} with shape {region.shape} "
        f"and requirements {shape_quantity}",
    )

Region 0 with shape (4, 4) and requirements [0 0 0 0 2 0]
Region 1 with shape (5, 12) and requirements [1 0 1 0 2 2]
Region 2 with shape (5, 12) and requirements [1 0 1 0 3 2]


In [181]:
# Upper bound: Perfect packing
shape_area = shape.sum(axis=(1, 2))
packed_area = np.sum(shape_quantity_requirements * shape_area, axis=1)
region_area = np.array(
    [np.prod(region.shape) for region in region_requirements],
)
can_fit = packed_area <= region_area
print(can_fit.sum())


3


In [182]:
# Lower Bound: Bounding boxes
rect_areas = np.zeros(shape.shape[0], dtype=int)
for i, shape_array in enumerate(shape):
    ys, xs = np.where(shape_array)
    min_x, max_x = xs.min(), xs.max()
    min_y, max_y = ys.min(), ys.max()
    rect_areas[i] = (max_x - min_x + 1) * (max_y - min_y + 1)
rect_areas = np.array(rect_areas)
print(rect_areas)

[9 9 9 9 9 9]


In [183]:
packed_area = np.sum(shape_quantity_requirements * rect_areas, axis=1)
can_fit_lower = packed_area <= region_area
print(can_fit_lower.sum())

1
